In [1]:
!pip install transformers evaluate datasets > /dev/null

In [2]:
from datasets import load_dataset

In [3]:
my_set = load_dataset("text",data_files={
    "train":["/content/linux_play.txt"],
    "test":["/content/vim_play.txt"]
})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-0a5f1a0b1e89beb1/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
my_set

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 277
    })
    test: Dataset({
        features: ['text'],
        num_rows: 62
    })
})

In [9]:
my_set['train'][0:5]['text']

['BEGIN;',
 'Linux Playbook',
 '',
 'The command and scenarios has to be executed inside the',
 'Kali Docker image. The docker image is called linux_playg.']

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [10]:
dataset = my_set.map(lambda examples: tokenizer(examples["text"]), 
                     batched=True)

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 277
    })
    test: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 62
    })
})

In [17]:
space_dataset = load_dataset("csv", 
                       data_files="/content/space_titanic.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
space_dataset

DatasetDict({
    train: Dataset({
        features: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
        num_rows: 8693
    })
})

In [ ]:
space_dataset['train'].shuffle(seed=4).select(range(10))[:]

In [18]:
space_dataset = space_dataset.rename_column(original_column_name='Transported',
                                            new_column_name='Target')

In [19]:
space_dataset

DatasetDict({
    train: Dataset({
        features: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Target'],
        num_rows: 8693
    })
})

In [20]:
space_dataset.unique("Target")

{'train': [False, True]}

In [21]:
space_dataset.unique("CryoSleep")

{'train': [False, True, None]}

In [24]:
space_dataset['train'][0]

{'PassengerId': '0001_01',
 'HomePlanet': 'Europa',
 'CryoSleep': False,
 'Cabin': 'B/0/P',
 'Destination': 'TRAPPIST-1e',
 'Age': 39.0,
 'VIP': False,
 'RoomService': 0.0,
 'FoodCourt': 0.0,
 'ShoppingMall': 0.0,
 'Spa': 0.0,
 'VRDeck': 0.0,
 'Name': 'Maham Ofracculy',
 'Target': False}

In [27]:
def first_name_get(name_value):
  return {"firstName":name_value['Name'].split(' ')[0]}

In [36]:
def name_length(name_value):
  return {"nameLength":len(name_value['Name'])}

In [30]:
def filter_name_none(name_value):
  return name_value['Name'] is not None

In [31]:
space_filter = space_dataset.filter(filter_name_none)

Filter:   0%|          | 0/8693 [00:00<?, ? examples/s]

In [32]:
space_filter

DatasetDict({
    train: Dataset({
        features: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Target'],
        num_rows: 8493
    })
})

In [35]:
space_filter = space_filter.map(first_name_get)

In [38]:
space_filter = space_filter.map(name_length)

In [46]:
hextra = space_filter.filter(lambda row: row['nameLength'] < 10)

In [47]:
hextra

DatasetDict({
    train: Dataset({
        features: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Target', 'firstName', 'nameLength'],
        num_rows: 456
    })
})

In [50]:
hextra['train'].train_test_split(train_size=0.8,
                                 seed=4)

DatasetDict({
    train: Dataset({
        features: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Target', 'firstName', 'nameLength'],
        num_rows: 364
    })
    test: Dataset({
        features: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Target', 'firstName', 'nameLength'],
        num_rows: 92
    })
})

In [54]:
type(space_filter)

datasets.dataset_dict.DatasetDict

In [58]:
space_filter['train']

AttributeError: ignored

In [59]:
!pip install txtinstruct langchain > /dev/null

In [60]:
from txtinstruct.models import StatementGenerator

# Load SQuAD dataset
dataset = load_dataset("squad", 
                       split="train")

# Train model
generator = StatementGenerator()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [61]:
from txtai.embeddings import Embeddings
from txtinstruct.data import DatasetBuilder

In [62]:
stmt_model, stmt_tokenizer = generator(
    "google/flan-t5-small",
    dataset,
    "sequence-sequence",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=128 // 16,
    num_train_epochs=0.01,
    logging_steps=100,
)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-73577150cf9f0b45/0.0.0. Subsequent calls will reuse this data.


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [63]:
from txtai.pipeline import Sequences

# Load statement generation model
statements = Sequences((stmt_model, 
                        stmt_tokenizer))

In [64]:
# Query templates
templates = [
    "Tell me about {text}",
    "Give an explanation on {text}",
    "Provide a quick summary on {text}",
    "Explain {text} in simple terms",
    "Describe {text}"
]

In [65]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)

In [66]:
!wget https://www.gutenberg.org/cache/epub/1/pg1.txt

--2023-05-10 05:39:58--  https://www.gutenberg.org/cache/epub/1/pg1.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120946 (118K) [text/plain]
Saving to: ‘pg1.txt’

pg1.txt             100%[===================>] 118.11K  --.-KB/s    in 0.04s   

2023-05-10 05:39:58 (2.68 MB/s) - ‘pg1.txt’ saved [120946/120946]



In [67]:
with open('pg1.txt','r') as d:
  data = d.read()

In [68]:
# Load embeddings
embeddings = Embeddings()

In [ ]:
embeddings.create()